## Import

In [2]:
#import libraries
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight
import numpy as np
import matplotlib.pyplot as plt
import keras_tuner
from keras.callbacks import EarlyStopping
from sklearn.metrics import roc_curve, roc_auc_score

In [3]:
#import data
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")
dataset = pd.concat([train,test])
dataset.head()

,id,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,0,L50096,L,300.6,309.6,1596,36.1,140,0.0,0,0,0,0,0
1,1,M20343,M,302.6,312.1,1759,29.1,200,0.0,0,0,0,0,0
2,2,L49454,L,299.3,308.5,1805,26.5,25,0.0,0,0,0,0,0
3,3,L53355,L,301.0,310.9,1524,44.3,197,0.0,0,0,0,0,0
4,4,M24050,M,298.0,309.0,1641,35.4,34,0.0,0,0,0,0,0


## Data Preprocessing

### Handling duplicates

In [4]:
#DUPLICATES
train.drop_duplicates(subset=train.columns.difference(['id']),inplace=True)
test.drop_duplicates(subset=test.columns.difference(['id']), inplace=True)

### Handling missing values

In [5]:
#MISSING VALUES
train.dropna(inplace=True)
test.dropna(inplace=True)

### Encode categorical features

In [6]:
#Searching for non-ordinal categorical features
categorical_columns = train.select_dtypes(include=['object']).columns.values
#Calculating unique values of categorical features
for col in categorical_columns:
    print(f" train {col}.unique = {len(train[col].unique())}, test {col}.unique = {len(test[col].unique())}") 

 train Product ID.unique = 9976, test Product ID.unique = 9909
 train Type.unique = 3, test Type.unique = 3


Since Product Id has too many unique values we cannot use one-hot encoding for this categorical feature because that will increase the dimesion of the feature space too much, resulting in slowing down the training time. Because of that we will use one-hot encoding for the Type feature and Frequency encoding for the ProductID feature (Some suggests to use Target enconding but I think that leads to overfitting)

In [7]:
#ONE-HOT ENCODING of Type column
for df in [train, test]:
    for value in df.Type.unique():
        df[f'Type{value}'] = 0
        df.loc[df.Type == f'{value}', f'Type{value}'] = 1
    df.drop(columns=['Type'], inplace=True)

In [8]:
#Frequency ENCODING of Product ID column (It is a way to utilize the frequency of the categories as labels)
for df in [train, test]:
    df['EncodedProductID'] = df.groupby(by=['Product ID'])['Product ID'].transform('count')
    df.drop(columns=['Product ID'], inplace=True)

## Feature engineering

In [9]:
def create_features(df):
    
    # Create a new feature by subtracting 'Air temperature' from 'Process temperature'
    # df['Temperature difference [K]'] = df['Process temperature [K]'] - df['Air temperature [K]']
    
    # Create a new feature by divided 'Air temperature' from 'Process temperature'
    df["Temperature ratio"] = df['Process temperature [K]'] / df['Air temperature [K]']
    
    # Create a new feature by multiplying 'Torque' and 'Rotational speed' (POWER)
    df['Torque * Rotational speed'] = df['Torque [Nm]'] * df['Rotational speed [rpm]']

    # Create a new feature by multiplying 'Torque' by 'Tool wear'
    df['Torque * Tool wear'] = df['Torque [Nm]'] * df['Tool wear [min]']

    # Create a new feature by adding 'Air temperature' and 'Process temperature'
    # df['Temperature sum [K]'] = df['Air temperature [K]'] + df['Process temperature [K]']
    
    # Create a new feature by multiplying 'Torque' by 'Rotational speed'
    df['Torque * Rotational speed'] = df['Torque [Nm]'] * df['Rotational speed [rpm]']

    df['TotalFailures'] = df[['TWF', 'HDF', 'PWF', 'OSF', 'RNF']].sum(axis=1)

    df.drop(['RNF'], axis =1, inplace = True)
    
    return df

In [10]:
train = create_features(train)
test = create_features(test)

## Model

In [11]:
class MyHyperModel(keras_tuner.HyperModel):
    def build(self, hp):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.InputLayer(input_shape=(n_features,)))
        model.add(tf.keras.layers.Dense(units=hp.Int("units1", min_value=8, max_value=512, step=16), activation='relu'))
        if hp.Boolean("BatchNormalization1", default=True):
            model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Dense(units=hp.Int("units2", min_value=32, max_value=512, step=32), activation='relu'))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Dense(units=hp.Int("units3", min_value=32, max_value=512, step=32), activation='relu'))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Dense(units=hp.Int("units4", min_value=32, max_value=512, step=32), activation='relu'))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Dense(units=hp.Int("units5", min_value=32, max_value=512, step=32), activation='relu'))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Dense(units=hp.Int("units6", min_value=32, max_value=512, step=32), activation='relu'))
        model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

        model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['AUC'])
        model.summary()

        return model
    
    def fit(self, hp, model, x, y, validation_data=None, **kwargs):
        return model.fit(
            x,
            y,
            batch_size = hp.Int("batch_size", min_value=8, max_value=512, step=8),
            validation_data=validation_data,
            **kwargs,
        )

In [12]:
train_X = train.drop(columns=['id', 'Machine failure']).reset_index(drop=True)
train_y = train['Machine failure'].reset_index(drop=True)
n_features = len(train_X.columns)

In [13]:
# Splitting train dataset into train and test
#train_X, test_X, train_y, test_y = train_test_split(train_X, train_y, test_size=0.33)

#using the actual test set
test_X = test.drop(columns=['id']).reset_index(drop=True)

In [14]:
train_X.shape

(135295, 17)

In [15]:
# StandardScaler
sc = StandardScaler() # MinMaxScaler or StandardScaler
train_X = sc.fit_transform(train_X)
test_X = sc.fit_transform(test_X)

In [16]:
# Splitting train dataset into train and val
train_X, val_X, train_y, val_y = train_test_split(train_X, train_y, test_size=0.33)

In [17]:
#weighting the unbalanced target
class_weights = dict(enumerate(class_weight.compute_class_weight(class_weight='balanced',
                                                 classes=np.unique(train_y),
                                                 y=train_y)))

In [18]:
tuner = keras_tuner.BayesianOptimization(
    hypermodel=MyHyperModel(),
    objective=keras_tuner.Objective("val_auc", direction="max"),
    max_trials=2,
    num_initial_points = 1,
    overwrite=True,
    directory="../hyperOptModelsHistory",
    project_name="BinaryClassificationofMachineFailure",
)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 144       
                                                                 
 batch_normalization (BatchN  (None, 8)                32        
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 32)                288       
                                                                 
 batch_normalization_1 (Batc  (None, 32)               128       
 hNormalization)                                                 
                                                                 
 dense_2 (Dense)             (None, 32)                1056      
                                                                 
 batch_normalization_2 (Batc  (None, 32)               1

In [19]:
tuner.search(train_X, train_y, epochs=10, validation_data=(val_X, val_y), class_weight=class_weights)


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
232               |232               |units1
False             |False             |BatchNormalization1
384               |384               |units2
352               |352               |units3
64                |64                |units4
352               |352               |units5
448               |448               |units6

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 232)               4176      
                                                                 
 dense_1 (Dense)             (None, 384)               89472     
                                                                 
 batch_normalization (BatchN  (None, 384)              1536      
 ormalization)                                                   
                                           

c:\Users\aless\miniconda3\envs\tf2.1-gpu\lib\site-packages\keras\backend.py:5673: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


11331/11331 [==============================] - 83s 7ms/step - loss: 0.4314 - auc: 0.9087 - val_loss: 0.1488 - val_auc: 0.9180
Epoch 2/10
11331/11331 [==============================] - 82s 7ms/step - loss: 0.4021 - auc: 0.9160 - val_loss: 0.2615 - val_auc: 0.7728
Epoch 3/10
11331/11331 [==============================] - 73s 6ms/step - loss: 0.3570 - auc: 0.9197 - val_loss: 0.0433 - val_auc: 0.9295
Epoch 4/10
 8753/11331 [======================>.......] - ETA: 18s - loss: 0.3091 - auc: 0.9283

In [ ]:
# Get the top 2 models.
models = tuner.get_best_models(num_models=2)
best_model = models[0]
# Build the model.
# Needed for `Sequential` without specified `input_shape`.
best_model.build(input_shape=(None, 28, 28))
best_model.summary()

In [ ]:
# Get the top 2 hyperparameters.
best_hps = tuner.get_best_hyperparameters(5)
# Build the model with the best hp.
model = build_model(best_hps[0])
# Fit with the entire dataset.
train_X = np.concatenate((train_X, val_X))
train_y = np.concatenate((train_y, val_y))

In [ ]:
history = model.fit(train_X, train_y, batch_size=512, epochs=1000, class_weight=class_weights, callbacks=[tf.keras.callbacks.EarlyStopping(patience=100)], validation_split=0.2)

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.legend()
plt.show()

In [ ]:
# Generate Prediction
y_pred = model.predict(val_X)
fpr, tpr, _ = roc_curve(val_y,  y_pred)
auc = roc_auc_score(val_y, y_pred)
plt.plot(fpr,tpr,label="auc="+str(auc))
plt.legend(loc=4)
plt.show()

In [ ]:
auc

In [ ]:
y_pred

In [ ]:
output = pd.DataFrame({
    "Machine failure" : np.squeeze(y_pred)
})
output

In [ ]:
output[output['Machine failure']>0.5].sort_values(by=['Machine failure'], ascending=False)